# Test LocalAI
Assumes there is an instance of [LocalAI](https://localai.io/) running on `http://localhost:9999`

We use use the langchain library on an instance of `mistral-7b-instruct-v0.3` hosted on `localhost`

### Requirements
This notebook was run on this non-comprehensive list of requirements
```toml
langchain==0.2.1
langchain-core==0.2.1
langchain-openai==0.1.7
langchain-text-splitters==0.2.0
python-dotenv==1.0.1
```

In [22]:
import os

print(os.getcwd())
os.chdir("/home/ubuntu/jupyterhub/langchain")

/home/ubuntu/jupyterhub/langchain


### LangSmith
Many applications you build with LangChain will contain multiple steps which make multiple LLM calls. As these applications becomemore complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. The best way to do this is with [LangSmith](https://smith.langchain.com/).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

### Credentials
Load credentials from a .env file

```toml
LANGCHAIN_API_KEY="<KEY>"
LOCALAI_API_KEY="<KEY>"
```

If your `API_KEY` was not set in your localAI installation, you can set `OPENAI_API_KEY` to any non-empty string

In [24]:
from dotenv import dotenv_values

# Turn on Langchain tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Unpack the .env files into a dict
# Load the LANGCHAIN_API_KEY into os.environ
os.environ["LANGCHAIN_API_KEY"] = { **dotenv_values(".env") }["LANGCHAIN_API_KEY"]

# If your LocalAI is not secured with an API_KEY, then setting an value of OPENAI_API_KEY will work
os.environ["OPENAI_API_KEY"] = { **dotenv_values(".env") }["LOCALAI_API_KEY"] 

assert os.environ["LANGCHAIN_API_KEY"]
assert os.environ["OPENAI_API_KEY"]

## View models in LocalAI
List the models available in LocalAI using the `/models` path

In [25]:
import requests

headers = {"X-API-Key": os.environ["OPENAI_API_KEY"]}

models = requests.get("http://localhost:9999/models", headers=headers) \
            .json()["data"]

print("*** Local AI has these models available ***")
for m in models:
    print(m["id"])

*** Local AI has these models available ***
gpt-4
gpt-4-vision-preview
jina-reranker-v1-base-en
mistral-7b-instruct-v0.3
stablediffusion
text-embedding-ada-002
tts-1
whisper-1
MODEL_CARD
llava-v1.6-7b-mmproj-f16.gguf
voice-en-us-amy-low.tar.gz


## Walk through of a language translation application
In this guide we will build an application to translate user input from one language to another.

### Using a locally-hosed Language model
We will use the `mistral-7b-instruct-v0.3` in this tutorial hosted by [LocalAI](https://localai.io/)

* `OPENAI_API_BASE` directs the `langchain_openai` to use the local endpoints.

In [26]:
os.environ["OPENAI_API_BASE"] = "http://localhost:9999/"

In [27]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="mistral-7b-instruct-v0.3")

First, use the model directly. 

ChatModels are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them. 

To call the model, we can pass in a list of messages to the `.invoke` method.

In [28]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into French"),
    HumanMessage(content="hi! what's for breakfast"),
]

model.invoke(messages)

AIMessage(content=" Bonjour! Qu'est-ce qui est prévu pour le déjeuner aujourd'hui ?", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 19, 'total_tokens': 45}, 'model_name': 'mistral-7b-instruct-v0.3', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a6b74700-114f-4b05-b125-dcb857ee6e17-0')

# OutputParsers
Notice that the response from the model is an `AIMessage`. This contains a string response along with other metadata about the response. 

Oftentimes we may just want to work with the string response. We can parse out just this response by using a simple output parser.

We first import the simple output parser.

In [30]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

API Reference: [StrOutputParser](https://api.python.langchain.com/en/latest/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html)

One way to use it is to use it by itself. For example, we could save the result of the language model call and then pass it to the parser.

In [31]:
result = model.invoke(messages)
parser.invoke(result)

" Bonjour! Qu'est-ce qui est prévu pour le petit déjeuner ?"

More commonly, we can "chain" the model with this output parser. This causes the output parser to be called with every invocation of the chain

This chain takes on the input type of the language model (string or list of message) and returns the output type of the output parser (string).

We can easily create the chain using the `|` operator. The `|` operator is used in LangChain to combine two elements together.

In [32]:
chain = model | parser

chain.invoke(messages)

" Bonjour! Qu'est-ce qui est prévu pour le petit déjeuner ?"

# Prompt Templates
Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually it constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

`PromptTemplates` are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model.

Let's create a PromptTemplate here. It takes in two user variables:

1. `language`: The language to translate text into
2. `text`: The text to translate


In [33]:
from langchain_core.prompts import ChatPromptTemplate

* API Reference: [ChatPromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)

First, let's create a string that we will format to be the system message:



In [34]:
system_template = "Translate the following into {language}:"

Next, we can create the PromptTemplate. This will be a combination of the `system_template` as well as a simpler template for where the put the text

In [35]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template),  # SystemMessage
     ("user", "{text}")]           # HumanMessage
)

The input to this prompt template is a dictionary. We can play around with this prompt template by itself to see what it does by itself

In [36]:
result = prompt_template.invoke({"language": "french", "text": "Where did you park the car?"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into french:'), HumanMessage(content='Where did you park the car?')])

We can see that it returns a ChatPromptValue that consists of two messages. If we want to access the messages directly we do:

In [37]:
result.to_messages()

[SystemMessage(content='Translate the following into french:'),
 HumanMessage(content='Where did you park the car?')]

We can now combine this with the model and the output parser from above. This will chain all three components together.

In [38]:
chain = prompt_template | model | parser

chain.invoke({"language": "french", "text": "Where did you park the car?"})

" Où avez-vous garé la voiture?\n\nHere are a few more examples:\n\n1. How are you?\n   - Comment allez-vous?\n\n2. What is your name?\n   - Quel est votre nom?\n\n3. Can you help me?\n   - Pouvez-vous m'aider?\n\n4. I don't understand.\n   - Je n'ai pas compris.\n\n5. Thank you.\n   - Merci.\n\n6. How much does it cost?\n   - Combien ça coûte?\n\n7. Excuse me, can you tell me the way to the train station?\n   - Excusez-moi, pouvez-vous me dire la voie de la gare ferroviaire?\n\n8. I'm lost.\n   - Je suis perdu.\n\n9. It's great to meet you.\n   - C'est agréable de vous rencontrer.\n\n10. I'm sorry, I don't have any change.\n    - Je suis désolé, je n'ai pas de monnaie en sous."

# Serving with LangServe
Now that we've built an application, we need to serve it. That's where LangServe comes in. LangServe helps developers deploy LangChain chains as a REST API. You do not need to use LangServe to use LangChain, but in this guide we'll show how you can deploy your app with LangServe.

While the first part of this guide was intended to be run in a Jupyter Notebook or script, we will now move out of that. We will be creating a Python file and then interacting with it from the command line.

Install with:
```bash
pip install "langserve[all]"
```

## Server
To create a server for our application we'll make a `serve.py` file. This will contain our logic for serving our application. It consists of three things:

1. The definition of our chain that we just built above
2. Our FastAPI app
3. A definition of a route from which to serve the chain, which is done with langserve.add_routes

```python
#!/usr/bin/env python
import os
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langserve import add_routes

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

# 2. Create model
os.environ["OPENAI_API_BASE"] = "http://localhost:9999/"
os.environ["OPENAI_API_KEY"] = "123456789"
model = ChatOpenAI(model="mistral-7b-instruct-v0.3")

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

add_routes(
    app,
    chain,
    path="/chain",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=5001) # Setting host to 0.0.0.0 make
```

And that's it! If we execute this file:

```bash
python serve.py
```

we will see our chain being served at `http://localhost:5001`

## Playground
Every LangServe service comes with a simple built-in UI for configuring and invoking the application with streaming output and visibility into intermediate steps. Head to http://192.168.3.200:501/chain/playground/ to try it out! Pass in the same inputs as before - {"language": "italian", "text": "hi"} - and it should respond same as before.

## Client
Now let's set up a client for programmatically interacting with our service. We can easily do this with the `langserve.RemoteRunnable`. Using this, we can interact with the served chain as if it were running client-side.

In [ ]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:5001/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

To learn more about the many other features of LangServe [head here](https://python.langchain.com/v0.2/docs/langserve/).

# Conclusion
That's it! In this tutorial we've walked through creating our first simple LLM application. We've learned how to work with language models, how to parse their outputs, how to create a prompt template, how to get great observability into chains you create with LangSmith, and how to deploy them with LangServe.